## Predicting student exam scores. With Student Performance dataset on kaggle. We will EDA, then split the data and apply machine learning model (Linear Regression, Decision Tree, Random Forest, XGBoost, SVM) to comment which model is best for the data


In [1]:
#Import Libaries
import pandas as pd
import numpy as np
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt

### 1. Read data description and summary



In [2]:
with open("/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt", "r") as f:
    content = f.read()
print(content)  

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

### 2. load dataset and Explore data



In [3]:
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [4]:
#Shape dataset
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (1460, 81)
Test shape: (1459, 80)


In [5]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
import pandas as pd
import numpy as np

def eda_descriptive_stats(df):
    """
    Performs basic Exploratory Data Analysis (EDA) and provides quick conclusions 
    from df.describe() for numerical columns.

    Args:
        df (pd.DataFrame): The input dataset.
    
    Returns:
        pd.DataFrame: The rounded descriptive statistics table (df.describe()).
    """
    print("--- DATA OVERVIEW ---")
    print(f"Number of Rows (Entries): {df.shape[0]}")
    print(f"Number of Columns (Features): {df.shape[1]}")
    
    # Print df.info() first for data types and memory usage
    print("\nData Types of Columns:")
    df.info() 
    print("-------------------------")
    
    # Select only numerical columns
    df_numerical = df.select_dtypes(include=np.number)
    if df_numerical.empty:
        print("The dataset contains no numerical columns for statistical analysis.")
        return None

    # Get the descriptive statistics table
    stats_df = df_numerical.describe().T
    
    # Round for easier reading
    stats_df = stats_df.round(2)
    
    print("\n--- DESCRIPTIVE STATISTICS ANALYSIS (df.describe().T) ---")
    print(stats_df)
    print("\n=========================================================")
    print("--- QUICK CONCLUSIONS FOR NUMERICAL DATA ---")

    # The total number of rows in the original DataFrame
    total_rows = df.shape[0]

    for col in stats_df.index:
        count = stats_df.loc[col, 'count']
        mean = stats_df.loc[col, 'mean']
        std = stats_df.loc[col, 'std']
        min_val = stats_df.loc[col, 'min']
        median = stats_df.loc[col, '50%']
        q1 = stats_df.loc[col, '25%']
        q3 = stats_df.loc[col, '75%']
        max_val = stats_df.loc[col, 'max']
        
        # Calculate IQR for more precise outlier check
        iqr = q3 - q1
        
        conclusion = f"**Column '{col}':**\n"
        
        # 1. Check for Missing Values
        missing_count = total_rows - count
        if missing_count > 0:
            missing_percent = (missing_count / total_rows) * 100
            conclusion += f"  - ⚠️ **Missing Data:** {missing_count} values ({missing_percent:.2f}%). Needs imputation/handling.\n"
        
        # 2. Check Distribution (Skewness)
        if mean > median * 1.1: # Significant positive skew
            conclusion += f"  - ➡️ **Right Skew:** Mean ({mean}) > Median ({median}). Likely due to **high outliers**.\n"
        elif mean < median * 0.9: # Significant negative skew
            conclusion += f"  - ⬅️ **Left Skew:** Mean ({mean}) < Median ({median}). Likely due to **low outliers**.\n"
        else:
            conclusion += f"  - ⚖️ **Near Symmetric:** Mean ({mean}) ≈ Median ({median}). Distribution is relatively balanced.\n"
            
        # 3. Check for Outliers (using IQR rule for a better simple check)
        upper_bound_iqr = q3 + 1.5 * iqr
        lower_bound_iqr = q1 - 1.5 * iqr
        
        outlier_notes = []
        if max_val > upper_bound_iqr * 1.5: # Use 1.5*IQR bound check as a strong indicator
             outlier_notes.append(f"Max ({max_val}) is significantly beyond the expected upper bound. Strong **High Outlier** detected.")
        if min_val < lower_bound_iqr and min_val != 0: 
             outlier_notes.append(f"Min ({min_val}) is significantly below the expected lower bound. Possible **Low Outlier** or **invalid data**.")

        if outlier_notes:
            conclusion += f"  - ⚡️ **Outlier Check:** {' '.join(outlier_notes)}\n"
        
        # 4. Check Variability (Coefficient of Variation - CV is better, but simple range check used here)
        range_val = max_val - min_val
        if range_val > 0 and std / mean > 0.5: # Simple check: Std is high relative to the mean
            conclusion += f"  - 📈 **High Variability:** Standard Deviation ({std}) is high relative to the mean. Data is widely dispersed.\n"
        else:
             conclusion += f"  - 📉 **Moderate Variability:** Data is relatively stable (Std: {std}).\n"
        
        print(conclusion)
        print("-" * 30)
    
    return stats_df

# --- Example Usage (Using the same dummy data structure) ---

# 1. Create a dummy dataset (must define 'np' if not already done)
try:
    # This block ensures 'np' is available for the example
    import numpy as np
except ImportError:
    print("NumPy not imported. Please ensure 'import numpy as np' is at the top.")

# Convert ID (int64) to object/string so it's not included in numerical stats
train_df['Id'] = train_df['Id'].astype('object')

# 2. Run the analysis function
print("\n>>> ANALYSIS RESULTS FOR DF_TRAIN <<<")
results_df = eda_descriptive_stats(train_df)


>>> ANALYSIS RESULTS FOR DF_TRAIN <<<
--- DATA OVERVIEW ---
Number of Rows (Entries): 1460
Number of Columns (Features): 81

Data Types of Columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   object 
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   obje

In [7]:
#resolve missing data
# for column LotFrontage(type float) so i use imputation
median_lot_frontage = train_df['LotFrontage'].median()
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(median_lot_frontage)
print(f"value null after resolve: {train_df['LotFrontage'].isnull().sum()}")

value null after resolve: 0


since all houses have garages but the year the garage was built has null values ​​in some columns. I use the hypothetical method where those houses were built without garages and when they are renovated they will add a garage.

In [8]:
train_df['GarageYrBlt'] = train_df['GarageYrBlt'].fillna(train_df['YearRemodAdd'])
print(f"value null after resolve: {train_df['YearRemodAdd'].isnull().sum()}")

value null after resolve: 0


In [9]:
#resolve same LotFrontage, for MasVnrArea
median_lot_frontage = train_df['MasVnrArea'].median()
train_df['MasVnrArea'] = train_df['MasVnrArea'].fillna(median_lot_frontage)
print(f"value null after resolve: {train_df['MasVnrArea'].isnull().sum()}")

value null after resolve: 0


In [10]:
#data discrepancy handling
skewed_features = [
    'SalePrice', 'LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 
    'MasVnrArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
    'BsmtFullBath', 'BsmtHalfBath', 'HalfBath'
]

for col in skewed_features:
    if col == 'SalePrice':
        train_df['SalePrice_Log'] = np.log1p(train_df[col]) 
    else:
        train_df[col] = np.log1p(train_df[col])

In [11]:

cols_to_check = skewed_features.copy()
cols_to_check.remove('SalePrice') 
cols_to_check.append('SalePrice_Log') # Thay SalePrice bằng cột log mới

# Tính toán độ lệch mới
skewness_results = train_df[cols_to_check].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

print("--- NEW DEVIATION AFTER LOG TRANSFORMATION ---")
print(skewness_results)

--- NEW DEVIATION AFTER LOG TRANSFORMATION ---
PoolArea         14.348342
3SsnPorch         7.727026
MiscVal           5.165390
BsmtHalfBath      3.929022
ScreenPorch       3.147171
BsmtFinSF2        2.521100
EnclosedPorch     2.110104
HalfBath          0.565586
MasVnrArea        0.503014
BsmtFullBath      0.418782
WoodDeckSF        0.153379
SalePrice_Log     0.121222
OpenPorchSF      -0.023373
LotArea          -0.137263
BsmtFinSF1       -0.617774
dtype: float64


In [12]:
#There is already a valid SalePrice_log Skewness column so delete SalePrice
cols_to_drop = ['SalePrice'] 

train_df.drop(columns=cols_to_drop, inplace=True)

In [13]:
cols_to_engineer = [
    'PoolArea', '3SsnPorch', 'MiscVal', 'BsmtHalfBath', 
    'ScreenPorch', 'BsmtFinSF2', 'EnclosedPorch'
]

for col in cols_to_engineer:
   # 1. Create a Binary Flag
# For example: 'HasPool', 'Has3SsnPorch', etc.
# This variable indicates whether the feature exists or not (Value > 0)
    train_df[f'Has{col}'] = (train_df[col] > 0).astype(int)
    
   # 2. Handling 0 values in the original column (Optional)
# Since we have applied log1p in the previous step, we leave the transformed column as is.
# If we have not log transformed, we should ensure that the original column has been Logarithmically transformed
# to handle non-zero values in the best way.

# Note: The original column df[col] has been log1p transformed and has a new skew.
# This column is now only useful for distinguishing non-zero values.

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 88 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1460 non-null   object 
 1   MSSubClass        1460 non-null   int64  
 2   MSZoning          1460 non-null   object 
 3   LotFrontage       1460 non-null   float64
 4   LotArea           1460 non-null   float64
 5   Street            1460 non-null   object 
 6   Alley             91 non-null     object 
 7   LotShape          1460 non-null   object 
 8   LandContour       1460 non-null   object 
 9   Utilities         1460 non-null   object 
 10  LotConfig         1460 non-null   object 
 11  LandSlope         1460 non-null   object 
 12  Neighborhood      1460 non-null   object 
 13  Condition1        1460 non-null   object 
 14  Condition2        1460 non-null   object 
 15  BldgType          1460 non-null   object 
 16  HouseStyle        1460 non-null   object 


In [15]:
def fill_categorical_missing_with_none(df):
    """
    fill missing value for categorical. by None
    Args:
        df (pd.DataFrame): dataset.
    """
    
    categorical_cols_with_na = df.select_dtypes(include=['object', 'category']).columns

    cols_to_fill = [col for col in categorical_cols_with_na if df[col].isnull().any()]
    
    if not cols_to_fill:
        print(" No find categorical column missing value")
        return

    print("--- Fill None for categorical column ---")
    print(f"Total column categorical need fill missing value: {len(cols_to_fill)}")

    for col in cols_to_fill:
        df[col].fillna("None", inplace=True)
        print(f"   Filled '{col}' bằng 'None'.")
        
    print("\nDone, check again:")
    print(df[cols_to_fill].isnull().sum().sort_values(ascending=False).head())

fill_categorical_missing_with_none(train_df)

--- Fill None for categorical column ---
Total column categorical need fill missing value: 16
   Filled 'Alley' bằng 'None'.
   Filled 'MasVnrType' bằng 'None'.
   Filled 'BsmtQual' bằng 'None'.
   Filled 'BsmtCond' bằng 'None'.
   Filled 'BsmtExposure' bằng 'None'.
   Filled 'BsmtFinType1' bằng 'None'.
   Filled 'BsmtFinType2' bằng 'None'.
   Filled 'Electrical' bằng 'None'.
   Filled 'FireplaceQu' bằng 'None'.
   Filled 'GarageType' bằng 'None'.
   Filled 'GarageFinish' bằng 'None'.
   Filled 'GarageQual' bằng 'None'.
   Filled 'GarageCond' bằng 'None'.
   Filled 'PoolQC' bằng 'None'.
   Filled 'Fence' bằng 'None'.
   Filled 'MiscFeature' bằng 'None'.

Done, check again:
Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
dtype: int64


/tmp/ipykernel_13/2977256252.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna("None", inplace=True)


In [16]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 88 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1460 non-null   object 
 1   MSSubClass        1460 non-null   int64  
 2   MSZoning          1460 non-null   object 
 3   LotFrontage       1460 non-null   float64
 4   LotArea           1460 non-null   float64
 5   Street            1460 non-null   object 
 6   Alley             1460 non-null   object 
 7   LotShape          1460 non-null   object 
 8   LandContour       1460 non-null   object 
 9   Utilities         1460 non-null   object 
 10  LotConfig         1460 non-null   object 
 11  LandSlope         1460 non-null   object 
 12  Neighborhood      1460 non-null   object 
 13  Condition1        1460 non-null   object 
 14  Condition2        1460 non-null   object 
 15  BldgType          1460 non-null   object 
 16  HouseStyle        1460 non-null   object 


### 3.Correlation Analysis between variables. The goal is to find important independent variables (features) that have a strong relationship with the target variable SalePrice_Log and detect Multicollinearity between features.


In [17]:
# Filter numeric columns that have been transformed
# Make sure to only get columns with data types of int or float
numeric_features = train_df.select_dtypes(include=['int', 'float']).columns

# Correlation of numeric columns with log-transformed target variable
correlation_with_target = train_df[numeric_features].corr()['SalePrice_Log'].sort_values(ascending=False)

#Remove SalePrice_Log (correlation is always 1) and display the result
print("--- TOP 10 BIẾN SỐ CÓ TƯƠNG QUAN MẠNH NHẤT VỚI SALECPRICE_LOG ---")
print(correlation_with_target.drop('SalePrice_Log').head(11))

--- TOP 10 BIẾN SỐ CÓ TƯƠNG QUAN MẠNH NHẤT VỚI SALECPRICE_LOG ---
OverallQual     0.817185
GrLivArea       0.700927
GarageCars      0.680625
GarageArea      0.650888
TotalBsmtSF     0.612134
1stFlrSF        0.596981
FullBath        0.594771
YearBuilt       0.586570
YearRemodAdd    0.565608
TotRmsAbvGrd    0.534422
GarageYrBlt     0.517069
Name: SalePrice_Log, dtype: float64


In [18]:
# Get the original correlation matrix (with signs) for relationship direction
corr_matrix_full = train_df[numeric_features].corr() 

# Get the absolute correlation matrix
corr_matrix_abs = corr_matrix_full.abs()

# Create an upper triangle matrix (removes duplicates and the diagonal)
upper_triangle = corr_matrix_abs.where(np.triu(np.ones(corr_matrix_abs.shape), k=1).astype(bool))

high_corr_pairs = []

# Iterate through each column (i) and columns after it (j)
for i in range(len(upper_triangle.columns)):
    for j in range(i + 1, len(upper_triangle.columns)):
        col_i = upper_triangle.columns[i]
        col_j = upper_triangle.columns[j]
        
        # Get the absolute correlation value
        correlation_value = upper_triangle.iloc[i, j]
        
        # Check the threshold (e.g., > 0.8)
        if correlation_value > 0.8:
            # Get the signed correlation value from the full matrix to know the direction
            signed_corr = corr_matrix_full.iloc[i, j]
            high_corr_pairs.append((col_i, col_j, correlation_value, signed_corr))

# ----------------- NEW ADDED LOGIC -----------------
# 1. Calculate the Absolute Correlation of ALL numerical variables with SalePrice_Log
# Absolute correlation measures the STRENGTH of the influence
target_corr_series = train_df[numeric_features].corr()['SalePrice_Log'].abs() 

print("--- LIST OF HIGHLY CORRELATED PAIRS (ABOVE 0.8) ---")

if high_corr_pairs:
    # Sort by correlation strength (absolute value) in descending order
    high_corr_pairs.sort(key=lambda x: x[2], reverse=True) 
    
    for c1, c2, abs_rho, signed_rho in high_corr_pairs:
        # Get absolute correlation with the target variable
        target_corr_c1 = target_corr_series.get(c1, 0)
        target_corr_c2 = target_corr_series.get(c2, 0)
        
        # Determine which variable should be kept (stronger correlation with SalePrice_Log)
        decision = ""
        if target_corr_c1 > target_corr_c2:
            decision = f" -> KEEP {c1} (Stronger)"
        elif target_corr_c2 > target_corr_c1:
            decision = f" -> KEEP {c2} (Stronger)"
        # Note: If correlations are equal, the tiebreaker is arbitrary (e.g., keeping C1)
        
        print(f"✅ Pair: {c1} & {c2}")
        print(f"   |Rho| (C1, C2): {abs_rho:.3f} (Signed Correlation: {signed_rho:.3f})")
        print(f"   Correlation with SalePrice_Log (Absolute):")
        print(f"      - {c1}: {target_corr_c1:.3f}")
        print(f"      - {c2}: {target_corr_c2:.3f}{decision}")

else:
    print("No independent variable pairs with correlation above 0.8 were found.")

--- LIST OF HIGHLY CORRELATED PAIRS (ABOVE 0.8) ---
✅ Pair: PoolArea & HasPoolArea
   |Rho| (C1, C2): 1.000 (Signed Correlation: 1.000)
   Correlation with SalePrice_Log (Absolute):
      - PoolArea: 0.070
      - HasPoolArea: 0.070 -> KEEP PoolArea (Stronger)
✅ Pair: ScreenPorch & HasScreenPorch
   |Rho| (C1, C2): 0.997 (Signed Correlation: 0.997)
   Correlation with SalePrice_Log (Absolute):
      - ScreenPorch: 0.106
      - HasScreenPorch: 0.099 -> KEEP ScreenPorch (Stronger)
✅ Pair: BsmtHalfBath & HasBsmtHalfBath
   |Rho| (C1, C2): 0.996 (Signed Correlation: 0.996)
   Correlation with SalePrice_Log (Absolute):
      - BsmtHalfBath: 0.005
      - HasBsmtHalfBath: 0.004 -> KEEP BsmtHalfBath (Stronger)
✅ Pair: 3SsnPorch & Has3SsnPorch
   |Rho| (C1, C2): 0.994 (Signed Correlation: 0.994)
   Correlation with SalePrice_Log (Absolute):
      - 3SsnPorch: 0.059
      - Has3SsnPorch: 0.058 -> KEEP 3SsnPorch (Stronger)
✅ Pair: EnclosedPorch & HasEnclosedPorch
   |Rho| (C1, C2): 0.992 (Signe

#### Việc tạo các biến nhị phân để tinh gọn lại giá trị của các biến có độ thiên lệch dữ liệu.Nhưng cả 2 biến đều có mức ý nghĩa tương tự như nhau nếu giữ lại sẽ làm bị hiện tượng đa cộng tuyến khiến mô hình hồi quy không thể xác định được mức độ đóng góp độc lập của các biến vì vậy sẽ giữ lại các biến đã được xử lí nhé.
#### Đối với các biến độc lập thì giữ lại biến nào có độ tương quan độc lập cao với biến SalePrice_Log(biến được xử lí thiên lệch dữ liệu về 1 phía)
#### Trường hợp một biến độc lập có mức độ tương quan cao đến biến SalePrice_Log vẫn giữ lại nhé.

In [19]:
import pandas as pd

def remove_weaker_multicollinear_features(df):
    """
    Identifies and removes the weaker correlated features within highly correlated 
    pairs (rho > 0.8) to address multicollinearity and redundancy. 
    
    The logic prioritizes keeping the Binary (HasX) feature over the Log-transformed 
    original feature for interpretability.

    Args:
        df (pd.DataFrame): The cleaned and transformed DataFrame.
    
    Returns:
        pd.DataFrame: The streamlined DataFrame.
    """
    # Create a copy to prevent the SettingWithCopyWarning
    df_cleaned = df.copy() 
    
    # --------------------------------------------------------------------------
    # 1. LIST OF COLUMNS TO DROP (Based on agreed-upon logic)
    # --------------------------------------------------------------------------
    
    print("--- IDENTIFYING COLUMNS FOR REMOVAL ---")
    
    # Group 1: True Multicollinearity (Keeping the stronger feature)
    # Strategy: KEEP GarageCars, GrLivArea, TotalBsmtSF
    cols_drop_multicollinearity = [
        'GarageArea',      # Weaker than GarageCars (0.651 < 0.681)
        'TotRmsAbvGrd',    # Weaker than GrLivArea (0.534 < 0.701)
        '1stFlrSF'         # Weaker than TotalBsmtSF (0.597 < 0.612)
    ]
    
    # Group 2: Log-transformed vs. Binary Pairs (Highly correlated)
    # Strategy: DROP the original Log-transformed column, KEEP the Binary ('HasX') 
    # feature for better categorical interpretability/modeling.
    
    cols_drop_log_binary = [
        'PoolArea',         # Dropping original log (Keeping HasPoolArea)
        'ScreenPorch',      # Dropping original log (Keeping HasScreenPorch)
        'BsmtHalfBath',     # Dropping original log (Keeping HasBsmtHalfBath)
        '3SsnPorch',        # Dropping original log (Keeping Has3SsnPorch)
        'EnclosedPorch',    # Dropping original log (Keeping HasEnclosedPorch)
        'MiscVal',          # Dropping original log (Keeping HasMiscVal)
        'BsmtFinSF2'        # Dropping original log (Keeping HasBsmtFinSF2)
    ]
    
    # Combine the lists
    cols_to_drop_final = cols_drop_multicollinearity + cols_drop_log_binary
    
    # Drop the original target variable (if present)
    if 'SalePrice' in df_cleaned.columns:
        cols_to_drop_final.append('SalePrice')

    # --------------------------------------------------------------------------
    # 2. EXECUTE COLUMN DROPPING
    # --------------------------------------------------------------------------
    
    # Filter for columns that actually exist in the DataFrame
    actual_cols_to_drop = [col for col in cols_to_drop_final if col in df_cleaned.columns]

    if actual_cols_to_drop:
        # Perform the drop operation
        df_cleaned = df_cleaned.drop(columns=actual_cols_to_drop)
        
        print("--- COLUMN REMOVAL COMPLETE (Prioritizing Binary Features) ---")
        print(f"✅ Columns successfully dropped: {actual_cols_to_drop}")
        print(f"Remaining column count: {df_cleaned.shape[1]}")
    else:
        print("No identified columns were found in the DataFrame to drop.")
        
    return df_cleaned

# --- EXAMPLE USAGE ---
# Assumption: 'train_df' is your loaded DataFrame
train_df_streamlined = remove_weaker_multicollinear_features(train_df)

--- IDENTIFYING COLUMNS FOR REMOVAL ---
--- COLUMN REMOVAL COMPLETE (Prioritizing Binary Features) ---
✅ Columns successfully dropped: ['GarageArea', 'TotRmsAbvGrd', '1stFlrSF', 'PoolArea', 'ScreenPorch', 'BsmtHalfBath', '3SsnPorch', 'EnclosedPorch', 'MiscVal', 'BsmtFinSF2']
Remaining column count: 78


In [20]:
train_df_streamlined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 78 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1460 non-null   object 
 1   MSSubClass        1460 non-null   int64  
 2   MSZoning          1460 non-null   object 
 3   LotFrontage       1460 non-null   float64
 4   LotArea           1460 non-null   float64
 5   Street            1460 non-null   object 
 6   Alley             1460 non-null   object 
 7   LotShape          1460 non-null   object 
 8   LandContour       1460 non-null   object 
 9   Utilities         1460 non-null   object 
 10  LotConfig         1460 non-null   object 
 11  LandSlope         1460 non-null   object 
 12  Neighborhood      1460 non-null   object 
 13  Condition1        1460 non-null   object 
 14  Condition2        1460 non-null   object 
 15  BldgType          1460 non-null   object 
 16  HouseStyle        1460 non-null   object 
